## Importation des modules

In [1]:
import pandas as pd # pour la manipulation de dataframes
import seaborn as sns # pour la visualisation
import matplotlib.pyplot as plt # pour la visualisation
import numpy as np # manipulation de tableaux
from sklearn.model_selection import train_test_split    # pour diviser les données en train et test sets
np.random.seed(42) # pour la reproductbilité des opérations aléatoires
from mord import LogisticAT
from typing import List
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from mord import LogisticAT, OrdinalRidge
import pickle
from sklearn.linear_model import LassoLarsIC
from sklearn.preprocessing import StandardScaler

# Lecture de la base et traitement des données

## Importation des données

In [2]:
df_aliments = pd.read_csv("C://Cours M2 S1//Conférences, SAS, VBA, Gestion de projet//Gestion de projet digital//data_clean_principal.csv")
df_aliments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52257 entries, 0 to 52256
Data columns (total 33 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   energy_100g                              52257 non-null  float64
 1   fat_100g                                 52257 non-null  float64
 2   saturated-fat_100g                       52257 non-null  float64
 3   trans-fat_100g                           52084 non-null  float64
 4   cholesterol_100g                         52164 non-null  float64
 5   carbohydrates_100g                       52257 non-null  float64
 6   sugars_100g                              52257 non-null  float64
 7   fiber_100g                               52257 non-null  float64
 8   proteins_100g                            52257 non-null  float64
 9   salt_100g                                52257 non-null  float64
 10  sodium_100g                              52257

## Nettoyage des données

On va retirer les colonnes avec trop de données manquantes pour éviter une trop grande perte de données

In [3]:
# suppresion des colonnes qui ne nous intéresse pas
df_aliments.drop(df_aliments.columns[15:32], axis=1, inplace=True)
# suppression des na restants
df_aliments.dropna(inplace=True)
# suppression des doublons
df_aliments.drop_duplicates(inplace=True)

In [4]:
df_aliments.info() # informations globales sur le dataframe

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52082 entries, 173 to 52256
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   energy_100g         52082 non-null  float64
 1   fat_100g            52082 non-null  float64
 2   saturated-fat_100g  52082 non-null  float64
 3   trans-fat_100g      52082 non-null  float64
 4   cholesterol_100g    52082 non-null  float64
 5   carbohydrates_100g  52082 non-null  float64
 6   sugars_100g         52082 non-null  float64
 7   fiber_100g          52082 non-null  float64
 8   proteins_100g       52082 non-null  float64
 9   salt_100g           52082 non-null  float64
 10  sodium_100g         52082 non-null  float64
 11  vitamin-a_100g      52082 non-null  float64
 12  vitamin-c_100g      52082 non-null  float64
 13  calcium_100g        52082 non-null  float64
 14  iron_100g           52082 non-null  float64
 15  nutrition_grade_fr  52082 non-null  object 
dtypes:

## Récupération des données de type numérique + variable d'intérêt (nutriscore)

In [5]:
# On code chaque nutriscore en attribuant la plus grande valeur à a (4), puis b (3)...
l = list(df_aliments["nutrition_grade_fr"].unique())
l.sort(reverse=True)
nutri_code = {score.lower() : i for i, score in enumerate(l)}

# on enregistre les codes pour réutilisation
df_nutri_code = pd.DataFrame(list(nutri_code.items()), columns=['Lettre', 'Valeur'])

# passage des lesttres aux codes pour les scores
df_aliments["nutrition_grade_fr"] = df_aliments["nutrition_grade_fr"].apply(lambda x: nutri_code[x])
df_aliments.rename(columns={"nutrition_grade_fr" : "score"}, inplace=True)

## Equilibrage des données

In [6]:
# On compte le nombre d'observation par classe
counts = df_aliments['score'].value_counts()
# on récupère le min
min_samples = counts.min()

# on crée un nouveau dataframe pour contenir les données équilibrées
df_aliments_eq = pd.DataFrame()
# on fait uun rééchantillonage en fixant le nombre d'observations par classe au min calculé précédemment
for label in counts.index:
    subset = df_aliments[df_aliments['score'] == label].sample(min_samples, random_state=42) # chaque échantillon est tiré aléatoirement et de manière reproductible avec le random seed
    df_aliments_eq = pd.concat([df_aliments_eq, subset], axis=0) # on joint les données en lignes


## Séparation en données de train et de test

In [7]:
df_train, df_test = train_test_split(df_aliments_eq, train_size=0.8, random_state = 42)

In [8]:
def get_train_test_sets(train_data, test_data):
    """Fonction qui récupère les dataframes train et test et retourne les vecteurs utiles pour la phase
    d'entrainement et test.

    Args:
        train_data (DataFrame): données d'entrainement pré-traitées
        test_data (DataFrame): données de test pré-traitées

    Returns:
        tuple: X_train, X_test, y_train, y_test
    """
    
    X_train = train_data.drop(labels=["score"], axis=1).values
    y_train = train_data["score"].values

    X_test = test_data.drop(labels=["score"], axis=1).values
    y_test = test_data["score"].values
    
    ## On normalise les données
    from sklearn.preprocessing import StandardScaler
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, X_test, y_train, y_test

In [9]:
X_train, X_test, y_train, y_test = get_train_test_sets(train_data=df_train, test_data=df_test)

# Le modèle de régression logistique ordinale initial

### On regarde les coefficients et l'erreur de classification du modèle global initial

In [12]:
#On entraîne le modèle initial

def model_training(X_train, X_test, y_train, y_test):
    """Fonction pour entrainer le modèle de régression logistique ordinale

    Args:
        X_train (Array): 
        X_test (Array): 
        y_train (Array): 
        y_test (Array): 
    """

    # Maintenant, on utilise les variables sélectionnées pour la régression logistique ordinale avec mord
    model_ordinal = LogisticAT(alpha=0)  # On utilise alpha=0 pour ne pas faire de régularisation

    # On entraîne le modèle sur les données d'entraînement avec les variables sélectionnées
    model_ordinal.fit(X_train, y_train)

    # prédictions sur les données de test avec les variables sélectionnées
    predictions_ordinal = model_ordinal.predict(X_test)

    # Calcul de l'exactitude du modèle de régression logistique ordinale
    accuracy_ordinal = accuracy_score(y_test, predictions_ordinal)
    
    print("Coefficients du modèle de régression logistique :")
    print(model_ordinal.coef_)
   
    print("Précision du modèle sur les données de test : {:.2f}%".format(accuracy_ordinal * 100))

    return

In [13]:
#On l'applqiue et on montre ses coefficients et son erreur de classification
model_training(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

Coefficients du modèle de régression logistique :
[-8.69434819e-01 -8.68057472e-01 -2.52818495e+00 -6.25123757e-01
 -1.17822480e+00 -2.90400221e-02 -2.14414845e+00  1.16671310e+00
  3.17895485e-01 -1.00605940e+02  9.87390027e+01 -3.81372754e-02
 -7.46646146e-02 -3.59015259e-01  1.16658539e+00]
Précision du modèle sur les données de test : 73.92%


# Régression logistique ordinale, pénalisé par la norme L2 (Lasso)

## On entraîne le modèle

In [47]:
def selection_lasso(train_data, test_data, max_iter = 1000, seuil=0.2):
    """Fonction pour faire la sélection de variables en se basant sur un modèle Lasso.

    Args:
        train_data (Dataframe): données préparées pour le train
        test_data (DataFrame): données préparées pour le test
        max_iter (int, optional): Nombre d'itérations pour obtenir la convergence, par défaut 1000
        seuil (float, optional): Valeur seuil en valeur absolur au dessus de laquelle on sélectionn les variables. Par défaut 0.2.

    Returns:
        tuple: variables_lasso (Liste) noms des variables choisies, X_train_final (Array), X_test_final (Array), y_train(Array), y_test (Array)
        vecteurs des variables choisies
    """
    
    from sklearn.linear_model import LogisticRegression
    from sklearn.feature_selection import SelectFromModel
    
    X_train, X_test, y_train, y_test = get_train_test_sets(train_data=train_data, test_data=test_data)
    

    # modèle de régression logistique avec pénalité L1 (LASSO)
    model_lasso = LogisticRegression(penalty='l1', solver='saga', max_iter=max_iter, random_state=None)

    # SelectFromModel pour récupérer les paramètres du modèle
    sfm = SelectFromModel(estimator=model_lasso)
    sfm.fit(X_train, y_train)

    # indices des variables sélectionnées
    indices_lasso = sfm.get_support(indices=True)
    
    # noms des variables correspondantes dans les données
    variables = [train_data.columns[i] for i in indices_lasso]
    
    # coefficients des variables sélectionnées
    coefficients_lasso = sfm.estimator_.coef_.flatten()

    # noms et coefficients des variables
    print("#########################################################")
    print()
    print("Coefficients obtenus après Lasso:")
    for variable, coefficient in zip(variables, coefficients_lasso):
        print(f"{variable}: {coefficient}")

    # On pénalise en ne prenant que les coeffs différents au dessus du seuil passé en argument
    indices_choisis = [i for i, coef in zip(indices_lasso, coefficients_lasso) if abs(coef) > seuil]

    # variables correspondantes aux indices choisies
    variables_lasso = [train_data.columns[i] for i in indices_choisis]

    print("##########################################################")
    print()
    print("Variables sélectionnées après Lasso")
    print()
    for variable in variables_lasso:
        print(f"{variable}")


    # on utilise les indices filtrés pour obtenir les données des variables optimales sélectionnées
    X_train_final = X_train[:, indices_choisis]
    X_test_final = X_test[:, indices_choisis]
    
    return variables_lasso, X_train_final, X_test_final, y_train, y_test 

## On effectue la régression Lasso optimale en les variables qu'elle a décidé de garder

In [48]:
_, X_train_lasso, X_test_lasso, y_train, y_test = selection_lasso(train_data=df_train, test_data=df_test)

#########################################################

Coefficients obtenus après Lasso:
energy_100g: 1.4809457185482384
fat_100g: 1.1784745640328433
saturated-fat_100g: 5.069634131411031
trans-fat_100g: 1.5000700149266886
cholesterol_100g: 1.737236853564534
carbohydrates_100g: 0.6348773999753657
sugars_100g: 2.9465576636099837
fiber_100g: -1.9580877559058576
proteins_100g: 0.0
salt_100g: 1.2797408152624574
sodium_100g: 1.2803935971849418
vitamin-a_100g: 0.8938012911289186
vitamin-c_100g: 0.5016039667485005
calcium_100g: 0.0
iron_100g: -7.484948492028028
##########################################################

Variables sélectionnées après Lasso

energy_100g
fat_100g
saturated-fat_100g
trans-fat_100g
cholesterol_100g
carbohydrates_100g
sugars_100g
fiber_100g
salt_100g
sodium_100g
vitamin-a_100g
vitamin-c_100g
iron_100g


C:\Users\alexa\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


## On l'entraîne pour nous donné son erreur de classification

In [49]:
def model_training(X_train, X_test, y_train, y_test, selection_method = 'lasso'):
    """Fonction pour entrainer le modèle de régression logistique ordinale

    Args:
        X_train (Array): 
        X_test (Array): 
        y_train (Array): 
        y_test (Array): 
    """

    # Maintenant, on utilise les variables sélectionnées pour la régression logistique ordinale avec mord
    model_ordinal = LogisticAT(alpha=0)  # On utilise alpha=0 pour ne pas faire de régularisation

    # On entraîne le modèle sur les données d'entraînement avec les variables sélectionnées
    model_ordinal.fit(X_train, y_train)

    # prédictions sur les données de test avec les variables sélectionnées
    predictions_ordinal = model_ordinal.predict(X_test)

    # Calcul de l'exactitude du modèle de régression logistique ordinale
    accuracy_ordinal = accuracy_score(y_test, predictions_ordinal)

   
    print("Précision du modèle sur les données de test : {:.2f}%".format(accuracy_ordinal * 100))

    return


In [50]:
model_training(X_train=X_train_lasso, X_test=X_test_lasso, y_train=y_train, y_test=y_test)

Précision du modèle sur les données de test : 72.58%


# Méthodes de sélection de variables par critères AIC|BIC

In [51]:
def get_train_test_sets(train_data, test_data):
    """Fonction qui récupère les dataframes train et test et retourne les vecteurs utiles pour la phase
    d'entrainement et test.

    Args:
        train_data (DataFrame): données d'entrainement pré-traitées
        test_data (DataFrame): données de test pré-traitées

    Returns:
        tuple: X_train, X_test, y_train, y_test
    """
    
    X_train = train_data.drop(labels=["score"], axis=1).values
    y_train = train_data["score"].values

    X_test = test_data.drop(labels=["score"], axis=1).values
    y_test = test_data["score"].values
    

    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = get_train_test_sets(train_data=df_train, test_data=df_test)

In [52]:
liste_variables = ['energy_100g', 'fat_100g', 'saturated-fat_100g', 'trans-fat_100g', 'cholesterol_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'salt_100g', 'vitamin-c_100g', 'calcium_100g', 'iron_100g', 'score']

df_train[liste_variables]

,energy_100g,fat_100g,saturated-fat_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,salt_100g,vitamin-c_100g,calcium_100g,iron_100g,score
20492,1502.0,2.0,0.47,0.787663,0.000000,72.00,1.0,0.000000,1.0000,0.019264,0.230625,0.007938,2
14016,788.0,7.1,2.20,0.198759,0.000563,20.00,2.3,2.700000,1.0000,0.028080,0.132071,0.005067,3
39163,1234.0,31.3,8.50,0.020546,0.015637,18.75,1.1,1.500000,1.8000,0.031769,0.710842,0.007000,1
5394,205.0,2.0,0.00,0.007910,0.000000,9.10,2.0,0.000000,0.1300,0.102756,0.015593,0.007031,4
25451,508.0,5.0,0.50,0.058824,0.028896,11.00,4.0,0.000000,1.8000,0.037283,0.173588,0.007982,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37635,1983.0,21.0,4.70,0.576880,0.000000,25.80,28.0,3.600000,0.6000,0.000016,0.309437,0.004825,1
20225,1910.0,19.4,1.70,0.265686,0.000000,42.20,3.0,10.100000,1.9304,0.067192,0.309656,0.006700,2
48693,2386.0,36.0,22.00,0.097566,0.015394,56.00,55.0,1.333333,0.1900,0.053304,0.288824,0.006360,0
41378,2046.0,18.4,4.50,0.058257,0.000000,64.70,2.2,4.500000,3.8100,0.060823,0.567159,0.008957,1


In [53]:
X_train

array([[1.50200000e+03, 2.00000000e+00, 4.70000000e-01, ...,
        1.92638554e-02, 2.30625000e-01, 7.93840580e-03],
       [7.88000000e+02, 7.10000000e+00, 2.20000000e+00, ...,
        2.80800000e-02, 1.32071429e-01, 5.06666667e-03],
       [1.23400000e+03, 3.13000000e+01, 8.50000000e+00, ...,
        3.17688772e-02, 7.10842105e-01, 7.00000000e-03],
       ...,
       [2.38600000e+03, 3.60000000e+01, 2.20000000e+01, ...,
        5.33042169e-02, 2.88823529e-01, 6.36027515e-03],
       [2.04600000e+03, 1.84000000e+01, 4.50000000e+00, ...,
        6.08231047e-02, 5.67158730e-01, 8.95689655e-03],
       [2.01000000e+02, 0.00000000e+00, 0.00000000e+00, ...,
        3.30000000e-02, 1.91908714e-01, 5.50622407e-03]])

In [65]:
# Fonction pour calculer l'erreur de classification
def classification_error(y_true, y_pred):
    return 1 - accuracy_score(y_true, y_pred)


## On normalise les données
scaler = StandardScaler()
X_train_ = scaler.fit_transform(X_train)
X_test_ = scaler.transform(X_test)

model_bic = LassoLarsIC(criterion='bic')
model_bic.fit(X_train_, y_train)
alpha_bic_ = model_bic.alpha_

model_aic = LassoLarsIC(criterion='aic')
model_aic.fit(X_train_, y_train)
alpha_aic_ = model_aic.alpha_

# Variables sélectionnées avec critère BIC
indices_variables_bic = np.where(model_bic.coef_ != 0)[0]
variables_bic = df_train.columns[indices_variables_bic]

# Entraîner le modèle de régression logistique ordinale avec critère BIC
model_ordinal_bic = LogisticAT(alpha=0)
model_ordinal_bic.fit(X_train[:, indices_variables_bic], y_train)

# Prédire les valeurs sur l'ensemble de test
y_pred_bic = model_ordinal_bic.predict(X_test[:, indices_variables_bic])

# statistiques bic
r2_bic = r2_score(y_test, y_pred_bic)
mse_bic = mean_squared_error(y_test, y_pred_bic)
accuracy_bic = accuracy_score(y_test, y_pred_bic)

# Variables sélectionnées avec critère AIC
indices_variables_aic = np.where(model_aic.coef_ != 0)[0]
variables_aic = df_train.columns[indices_variables_aic]

# Entraîner le modèle de régression logistique ordinale avec critère AIC
model_ordinal_aic = LogisticAT(alpha=0)
model_ordinal_aic.fit(X_train[:, list(indices_variables_aic)], y_train)

# Prédire les valeurs sur l'ensemble de test
y_pred_aic = model_ordinal_aic.predict(X_test[:, list(indices_variables_aic)])

# statistiques aic
r2_aic = r2_score(y_test, y_pred_aic)
mse_aic = mean_squared_error(y_test, y_pred_aic)
accuracy_aic = accuracy_score(y_test, y_pred_aic)

# Calculer l'erreur de classification pour les modèles AIC et BIC
error_aic = classification_error(y_test, y_pred_aic)
error_bic = classification_error(y_test, y_pred_bic)

#Résultats
results = pd.DataFrame({
    "Critère": ["AIC", "BIC"],
    "Variables Sélectionnées": [list(variables_aic), list(variables_bic)],
    "R²": [r2_aic, r2_bic],
    "MSE": [mse_aic, mse_bic],
    "Accuracy model ordinal": [accuracy_aic, accuracy_bic],
    "Classification Error": [error_aic, error_bic],
    "Alpha (AIC)": [alpha_aic_, np.nan],  # Notez que la valeur de l'alpha pour BIC est obtenue de la même manière
    "Alpha (BIC)": [np.nan, alpha_bic_]
})

def highlight_min(s):
    is_min = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_min]

## On regarde les résultats

In [55]:
results.style.apply(highlight_min, subset=["Alpha (AIC)", "Alpha (BIC)"])

,Critère,Variables Sélectionnées,R²,MSE,Accuracy model ordinal,Classification Error,Alpha (AIC),Alpha (BIC)
0,AIC,"['energy_100g', 'fat_100g', 'saturated-fat_100g', 'trans-fat_100g', 'cholesterol_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g', 'sodium_100g', 'vitamin-a_100g', 'vitamin-c_100g', 'calcium_100g', 'iron_100g']",0.831980,0.334886,0.702388,0.297612,0.000000,nan
1,BIC,"['energy_100g', 'fat_100g', 'saturated-fat_100g', 'trans-fat_100g', 'cholesterol_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g', 'vitamin-a_100g', 'vitamin-c_100g', 'calcium_100g', 'iron_100g']",0.834902,0.329062,0.707513,0.292487,nan,0.000000


In [61]:
#Variables sélectionnées par le critère BIC
results["Variables Sélectionnées"][1]

['energy_100g',
 'fat_100g',
 'saturated-fat_100g',
 'trans-fat_100g',
 'cholesterol_100g',
 'carbohydrates_100g',
 'sugars_100g',
 'fiber_100g',
 'proteins_100g',
 'salt_100g',
 'vitamin-a_100g',
 'vitamin-c_100g',
 'calcium_100g',
 'iron_100g']

In [64]:
# Afficher les coefficients du modèle optimal au sens du BIC
coefficients_bic = model_ordinal_bic.coef_
print("Coefficients du modèle optimal au sens du BIC:")
for variable, coefficient in zip(variables_bic, coefficients_bic):
    print(f"{variable}: {coefficient}")

Coefficients du modèle optimal au sens du BIC:
energy_100g: -0.001210366538994659
fat_100g: -0.06318952650259746
saturated-fat_100g: -0.3926341463073361
trans-fat_100g: -1.7524813352994169
cholesterol_100g: -0.2738357542441615
carbohydrates_100g: -0.0024272228238948136
sugars_100g: -0.12955563238100962
fiber_100g: 0.4694123287145111
proteins_100g: 0.04014568266620962
salt_100g: -1.681459233553816
vitamin-a_100g: 0.0031130506281653015
vitamin-c_100g: 0.060925164001909546
calcium_100g: -0.5689939006249928
iron_100g: 1.0957672907789342


In [58]:
#Application du modèle
model = LogisticAT(alpha=alpha_bic_)
model.fit(X_train[:, list(indices_variables_bic)], y_train)

# Effectuez les prédictions sur les données de test
predictions = model.predict(X_test[:, indices_variables_bic])

# Calculez l'exactitude des prédictions
accuracy_ordinal = accuracy_score(y_test, predictions)

# Imprimez le pourcentage de précision
print("{:.2f}%".format(accuracy_ordinal * 100))

71.26%
